In [1]:
import re
import os
import json
import glob
from copy import deepcopy
from collections     import Counter

from tqdm import tqdm
import pandas as pd
import Levenshtein as lev
import numpy as np

from pybtex.database import parse_string
from pylatexenc.latex2text import LatexNodes2Text
conv = LatexNodes2Text()

In [2]:
import rispy
mappings = deepcopy(rispy.TAG_KEY_MAPPING)
mappings['M2'] = 'extra'
mappings['AB'] = 'orig_abstract'
mappings['SV'] = 'series_volume'

In [3]:
def extract_isbn(input_string):
    """
    Extract an ISBN from an unstructured text string.
    
    The function searches for ISBN numbers that may include a check character 'X' at the end.
    It recognizes ISBNs both with and without spaces or hyphens between segments.
    """
    # Regular expression to match "ISBN:" followed by any combination of digits, hyphens, and possibly ending with an 'X'
    pattern = r'ISBN:?\s*([\d\-]+X?)'
    
    match = re.search(pattern, input_string)
    if match:
        return match.group(1)  # Return the matched part (ISBN number)
    else:
        return None  # No ISBN found following the "ISBN:" prefix
    
import re

def extract_issn(input_string):
    """
    Extract an ISSN from an unstructured text string.
    
    The function searches for ISSN numbers, which are typically in the format '1234-5678'.
    It recognizes ISSNs both with and without spaces or hyphens between segments.
    """
    # Regular expression to match "ISSN:" followed by a valid ISSN format
    pattern = r'ISSN:?\s*(\d{4}-\d{3}[\dX])'
    
    match = re.search(pattern, input_string)
    if match:
        return match.group(1)  # Return the matched part (ISSN number)
    else:
        return None  # No ISSN found

In [4]:
extract_isbn("ISBN: 978-0-19-880393-5. P. VII-VIII Acknowledgements; p. XI-XII List of illustrations; p. XIII-XXIV Preface: what this book is (not) about; p. 1-21 Introduction: biblical philology in the sixteenth century; p. 253-280 Bibliography; p. 281-296 Index.")

'978-0-19-880393-5'

In [5]:
def map_chapter(ris, bibt):
    # make editors authors, if applicable:
    if 'editor' in bibt.persons:
        if 'author' in bibt.persons:
            ris['first_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['author']]
        else:
            if 'authors' in ris:
                del ris['authors']
        ris['secondary_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['editor']]

    if 'first_authors' in ris and 'secondary_authors' in ris:
        if 'authors' in ris:
            del ris['authors']

    # add translators
    if 'translator' in bibt.fields:
        ris['tertiary_authors'] = conv.latex_to_text(bibt.fields['translator']).split(' and ')

    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # replace non-distinct title with parsed journal title:
    if 'title' in bibt.fields:
        ris['extra'] = ris['title']
        ris['title'] = bibt.fields['title']

    if 'booktitle' in bibt.fields:
        ris['secondary_title'] = bibt.fields['booktitle']
    
    if 'pagetotal' in bibt.fields:
        ris['end_page'] = bibt.fields['pagetotal']
    
    if 'pages' in bibt.fields:
        pages = bibt.fields['pages'].split('-')
        if len(pages) == 2:
            ris['start_page'] = pages[0]
            ris['end_page'] = pages[1]
        else:
            ris['start_page'] = bibt.fields['pages']
    
    if 'publisher' in bibt.fields:
        ris['publisher'] = bibt.fields['publisher']

    if 'place' in bibt.fields:
        ris['place_published'] = bibt.fields['place']

    if 'address' in bibt.fields:
        ris['place_published'] = bibt.fields['address']
    
    if 'location' in bibt.fields:
        ris['place_published'] = bibt.fields['location']

    if 'series' in bibt.fields:
        ris['tertiary_title'] = bibt.fields['series']

    if 'volume' in bibt.fields:
        ris['volume'] = bibt.fields['volume']

    if 'number' in bibt.fields:
        ris['series_volume'] = bibt.fields['number']

    if 'edition' in bibt.fields:
        ris['edition'] = bibt.fields['edition']
    
    return ris

Collect already available normalizations for journal titles (so that we can find out below which one we miss):

In [6]:
jtitle = pd.read_excel('../data/journal_titles_master.xlsx')
existing_jtitles = set(jtitle['normalized'])
lower2jtitles = dict(zip(jtitle['normalized'].str.lower(), jtitle['normalized']))
jtitle.head(30)

,secondary_title,normalized,count,issn
0,Ons erfdeel: kultureel tijdschrift voor Zuidvl...,Ons erfdeel,2660.0,0030-2651
1,Dietsche warande en Belfort: tijdschrift voor ...,Dietsche warande en Belfort,2461.0,0012-2645
2,De nieuwe taalgids: tweemaandelijks tijdschrif...,De nieuwe taalgids,2359.0,0028-9922
3,Bzzlletin; Stichting BZZTôH Teater. Voorburg: ...,Bzzlletin,1638.0,0165-0858
4,Poëziekrant: tweemaandelijks tijdschrift. Gent...,Poëziekrant,1573.0,2030-0638
5,Onze taal: maandblad van het Genootschap Onze ...,Onze taal,1322.0,0165-7828
6,Vlaanderen: tweemaandelijks tijdschrift voor k...,Vlaanderen,1312.0,0042-7683
7,De gids: nieuwe vaderlandsche letteroefeningen...,De gids,1249.0,0016-9730
8,Levende talen: berichten en mededelingen van d...,Levende talen,1239.0,0024-1539
9,Tijdschrift voor Nederlandse taal- en letterku...,Tijdschrift voor Nederlandse taal- en letterkunde,962.0,0040-7550


In [7]:
def map_journal(ris, bibt):
    """
    Merges the newly structured information in the bibtex returned
    by the LLM into the already available RIS entry from the dump.
    Reliably structured information (e.g. authors, year, keywords, ...)
    from the RIS entries is maximally retained.
    """
    #print(ris)
    #print(bibt)
    #print('===============================================')

    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # replace unstructured title with parsed journal title (if available):
    if 'title' in bibt.fields:
        # keep track of original title description:
        ris['extra'] = [ris['title']]
        ris['title'] = bibt.fields['title']
        if ris['title'].endswith(','):
            ris['title'] = ris['title'][:-1]
        if ris['title'].strip().lower() in ('in', 'untitled', 'title of the article', 'title of the article (if provided)'):
            ris['title'] = ''
    
    # parse pagination information:
    if 'pagetotal' in bibt.fields:
        ris['end_page'] = bibt.fields['pagetotal']
    if 'pages' in bibt.fields:
        pages = bibt.fields['pages'].split('-')
        if len(pages) == 2:
            ris['start_page'] = pages[0]
            ris['end_page'] = pages[1]
        else:
            ris['end_page'] = bibt.fields['pages']

    # collect parsed journal title (unless we had that information already, which will be more reliable)
    if 'secondary_title' not in ris and 'journal' in bibt.fields:
        journal = bibt.fields['journal']
        # sometimes place of publication of the journal is added: we remove that
        journal = journal.split('(')[0].strip()
        journal = journal.split('[')[0].strip()
        ris['journal_name'] = journal
    elif 'secondary_title' in ris and 'journal' in bibt.fields:
        ris['journal_name'] = ris['secondary_title']
        del ris['secondary_title']

    if 'journal_name' in ris:
        jn = ris['journal_name']
        if jn.startswith('"') and jn.endswith('",'):
            jn = jn[1:-2]
        if jn.count('"') == 1:
            jn = jn.replace('"', '')
        ris['journal_name'] = jn
    
    # collect information on volume and issue
    if 'volume' in bibt.fields:
        ris['volume'] = bibt.fields['volume']
    if 'number' in bibt.fields:
        ris['number'] = bibt.fields['number']
    if 'number' not in bibt.fields and 'issue' in bibt.fields:
        ris['number'] = bibt.fields['issue']
    
    if 'volume' in ris and not 'number' in ris:
        ris['number'] = ris['volume']
        del ris['volume']
    
    return ris

In [8]:
def map_book(ris, bibt):
    # extract ISBN for abstract field, if available:
    if 'orig_abstract' in ris:
        abstract = ris['orig_abstract']
        isbn = extract_isbn(abstract.strip())
        if isbn:
            ris['issn'] = isbn

    # make editors authors, if applicable:
    if 'editor' in bibt.persons:
        if 'author' in bibt.persons:
            ris['first_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['author']]
        else:
            if 'author' in ris:
                del ris['author']
            if 'authors' in ris:
                del ris['authors']
        ris['tertiary_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['editor']]

    if 'first_authors' in ris and 'tertiary_authors' in ris:
        if 'author' in ris:
            del ris['author']
        if 'authors' in ris:
            del ris['authors']

    # add translators
    if 'translator' in bibt.fields:
        ris['subsidiary_authors'] = conv.latex_to_text(bibt.fields['translator']).split(' and ')
        if 'author' in ris:
            del ris['author']
        if 'first_authors' in ris:
            del ris['first_authors']

    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # replace non-distinct title with parsed book title:
    if 'title' in bibt.fields and bibt.fields['title']:
        try:
            ris['extra'].append(ris['title'])
        except KeyError:
            ris['extra'] = [ris['title']]
        ris['title'] = bibt.fields['title']
    elif 'booktitle' in bibt.fields and bibt.fields['booktitle']:
        try:
            ris['extra'].append(ris['title'])
        except KeyError:
            ris['extra'] = [ris['title']]
        ris['title'] = bibt.fields['booktitle']
    
    if 'pagetotal' in bibt.fields:
        ris['start_page'] = bibt.fields['pagetotal']
    
    if 'pages' in bibt.fields:
        ris['start_page'] = bibt.fields['pages']
    
    if 'publisher' in bibt.fields:
        ris['publisher'] = bibt.fields['publisher']

    if 'place' in bibt.fields:
        ris['place_published'] = bibt.fields['place']

    if 'place' in bibt.fields:
        place = bibt.fields['place']
        if ris['extra'] and '[' + place + ']' in ris['extra'][0]:
            place = '[' + place + ']'
        ris['place_published'] = place

    if 'address' in bibt.fields:
        ris['place_published'] = bibt.fields['address']
    
    if 'location' in bibt.fields:
        ris['place_published'] = bibt.fields['location']

    if 'series' in bibt.fields:
        if ';' in bibt.fields['series']:
            series, vol = [e.strip() for e in bibt.fields['series'].split(';', maxsplit=1)]
            ris['secondary_title'] = series
            ris['note'] = vol
        else:
            ris['secondary_title'] = bibt.fields['series']

    if 'volume' in bibt.fields:
        ris['note'] = bibt.fields['volume']

    if 'number' in bibt.fields:
        ris['note'] = bibt.fields['number']

    if 'edition' in bibt.fields:
        ris['edition'] = bibt.fields['edition']
    
    return ris

In [9]:
def map_jfull(ris, bibt):
    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # extract ISSN from title field, if available:
    if 'title' in ris:
        abstract = ris['title']
        issn = extract_issn(abstract.strip())
        if issn:
            ris['issn'] = issn
    
    # replace non-distinct title with parsed book title:
    if 'title' in bibt.fields and bibt.fields['title']:
        ris['extra'] = [ris['title']]
        ris['title'] = bibt.fields['title']
    elif 'booktitle' in bibt.fields and bibt.fields['booktitle']:
        ris['extra'] = ris['title']
        ris['title'] = bibt.fields['booktitle']

    # make editors authors, if applicable:
    if 'editor' in bibt.persons:
        if 'author' in bibt.persons:
            ris['first_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['author']]
        else:
            if 'author' in ris:
                del ris['author']
            if 'authors' in ris:
                del ris['authors']
        ris['tertiary_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['editor']]

    if 'first_authors' in ris and 'tertiary_authors' in ris:
        if 'author' in ris:
            del ris['author']
        if 'authors' in ris:
            del ris['authors']

    # add translators
    if 'translator' in bibt.fields:
        ris['subsidiary_authors'] = conv.latex_to_text(bibt.fields['translator']).split(' and ')
        if 'author' in ris:
            del ris['author']
        if 'first_authors' in ris:
            del ris['first_authors']
    
    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # pagination information:
    if 'pagetotal' in bibt.fields:
        ris['start_page'] = bibt.fields['pagetotal']
    if 'pages' in bibt.fields:
        ris['start_page'] = bibt.fields['pages']
    
    # publisher information:
    if 'publisher' in bibt.fields:
        ris['publisher'] = bibt.fields['publisher']

    # place of publication:
    if 'place' in bibt.fields:
        ris['place_published'] = bibt.fields['place']
    if 'place' in bibt.fields:
        place = bibt.fields['place']
        if ris['extra'] and '[' + place + ']' in ris['extra'][0]:
            place = '[' + place + ']'
        ris['place_published'] = place
    if 'address' in bibt.fields:
        ris['place_published'] = bibt.fields['address']
    if 'location' in bibt.fields:
        ris['place_published'] = bibt.fields['location']

    if 'volume' in bibt.fields:
        ris['volume'] = bibt.fields['volume']

    if 'number' in bibt.fields:
        ris['note'] = bibt.fields['number']

    # Hard reset to BOOK
    ris['type_of_reference'] = 'BOOK'

    if 'keywords' in ris:
        ris['keywords'] = list(set(ris['keywords']))
        ris['keywords'].append('Speciaal tijdschriftnummer')
    
    return ris

In [10]:
map_entry = {
             'JOUR': map_journal,
             'CHAP': map_chapter,
             'BOOK': map_book,
             'JFULL': map_jfull,
             'EJOUR': map_journal,
             'ADVS': map_book,
             'WEB': map_book,
            }

In [11]:
def deduplicate_bibtex(bibt):
    """
    Deduplicate repeated fields in the bibtex returned by the LLM.
    We only keep the first appearance of a given field.
    """
    lines, fields = [], set()
    for line in bibt.strip().split('\n'):
        if line.startswith('@') or line == '}':
            lines.append(line)
        else:
            field = line.split('=')[0].strip()
            if field not in fields:
                lines.append(line)
                fields.add(field)

    clean = '\n'.join([l for l in lines if l])
    if not clean.strip().endswith('}'):
        clean += '\n}\n'
    
    return clean


def clean_bibtex(bibt):
    """
    Attempts to correct some common syntactic errors in the bibtex
    returned by the LLM (which cause the pybtex parser to fail).
    """
    if not bibt:
        return ''
    
    # remove erroneous markdown syntax:
    bibt = bibt.replace('```bibtex', '').replace('```', '').replace("```tex", '')
    
    # sometimes mutliple bibtexs are created: we only keep the first one
    bibt = [b for b in bibt.split('@') if b.strip()]
    bibt = '@' + bibt[0]

    lines = []
    for line in bibt.strip().split('\n'):
        l = line.strip()

        # take care of spaces in the bibtex key:
        if l.startswith('@') and ' ' in l:
            line = ''.join(line.split())
        
        # fix missing entry keys:
        if l in ('@article{,', '@article{'):
            lines.append('@article{xxx,')
            continue
        if l in ('@book{,', '@book{'):
            lines.append('@book{xxx,')
            continue
        if l in ('@incollection{,', '@incollection{'):
            lines.append('@incollection{xxx,')
            continue

        # common errors:
        if l.endswith(']'):
            line += '},'
            lines.append(line)
            continue
        line = line.replace('{ )', '{}')
        if l.endswith("',"):
            line = line[:-2] + '},'
        
        # ensure that end-of-line syntax is respected:
        if l != '}':
            if not l.endswith('},'):
                if l.endswith('}'):
                    line += ','
                elif not l.endswith('}') and not l.endswith(','):
                    line += '},'
            if l.endswith('),'):
                line = line.replace('),', ')},')
        
        # add missing curly brackets:
        if '=' in l and (not '{' in l or not '}' in l):
            k, v = [e.strip() for e in l.split('=')][:2]
            v.replace(',', '')
            line = '  ' + k + '=' + '{' + v + '},'
        
        # remove lines with empty values:
        if '= {},' in l:
            continue

        if ' &' in line:
            line = line.replace(' &', ' \&')
        
        # correct curly bracket syntax in title field:
        if l.startswith('title') and l.count('}') > 1:
            k, v = [e.strip() for e in l.split('=')][:2]
            v = v.replace('{', '').replace('}', '')
            line = '  ' + k + '=' + '{' + v + '},'

        # correct syntax:
        if '",' in l and '=' in l:
            k, v = [e.strip() for e in l.split('=')][:2]
            if v.startswith('"') and v.endswith('",'):
                v = v[1:-2]
            line = '  ' + k + '=' + '{' + v + '},'

        lines.append(line)

    # recompose the lines of the bibtex entry:
    clean = '\n'.join([l for l in lines if l])
    if not clean.strip().endswith('}'):
        clean += '\n}\n'
    
    # return the deduplicated version of the bibtex entry:
    return deduplicate_bibtex(clean)

<>:87: SyntaxWarning: invalid escape sequence '\&'
<>:87: SyntaxWarning: invalid escape sequence '\&'
/var/folders/6j/pn9nz6b55j3fpdt8gtj50crm0000gn/T/ipykernel_84431/2234798805.py:87: SyntaxWarning: invalid escape sequence '\&'
  line = line.replace(' &', ' \&')


In [12]:
llm_path = '../data/llm-dump'

new_jtitles = Counter()

for decade_folder in sorted(glob.glob(f'{llm_path}/*')):
    #if '2010s' not in decade_folder:
    #        continue
    print(':::', decade_folder, ':::')

    for spreadsheet_path in sorted(glob.glob(f'{decade_folder}/*.xlsx')):
        df = pd.read_excel(spreadsheet_path, header=0, engine='openpyxl')
        #n = 5000
        #if len(df) > n:
        #    df = df.sample(n)

        if 'bibtex' not in df.columns:
            continue
    
        ptype = os.path.basename(spreadsheet_path).replace('.xlsx', '')
        print('     - ', spreadsheet_path, f'({ptype})')

        if ptype not in ('BOOK', 'JOUR', 'CHAP', 'JFULL', 'EJOUR', 'ADVS', 'WEB'):
        #if ptype != 'JFULL':
            continue
        
        # parse the RIS (stored as JSON strings in the spreadsheet)
        df['RIS'] = df['RIS'].apply(json.loads)

        # clean (and deduplicate the bibtex returned by the LLM)
        cleaned = []
        for bt in df['bibtex']:
            if isinstance(bt, str):
                cleaned.append(clean_bibtex(bt))
            else:
                cleaned.append('')
        df['bibtex-clean'] = cleaned

        # Update the available RIS entries with newly structure info,
        # returned by the LLM (and keep tracked of whether or not that is successful):
        updated_ris, status = [], []
        for ris, bibtex_str in tqdm(list(zip(df['RIS'], df['bibtex-clean']))):
            if 'abstract' in ris:
                ris['orig_abstract'] = ris['abstract']
                del ris['abstract']
            if isinstance(bibtex_str, str):
                try:
                    #print(bibtex_parse)
                    bibtex_parse = parse_string(bibtex_str, 'bibtex')
                    single_key = list(bibtex_parse.entries.keys())[0]
                    updated = map_entry[ptype](ris.copy(), bibtex_parse.entries[single_key])

                    # keep track of new journal titles which lack a normalized variant,
                    # (unless the difference is only in capitalization):
                    if ptype in 'JOUR' and 'journal_name' in updated and updated['journal_name'] not in existing_jtitles:
                        try:
                            updated['journal_name'] = lower2jtitles[updated['journal_name'].lower()]
                        except KeyError:
                            new_jtitles[updated['journal_name']] += 1
                    
                    updated['label'] = 'success'
                    if 'authors' not in updated and 'secondary_authors' not in updated and \
                        'primary_authors' not in updated and 'tertiary_authors' not in updated \
                        and 'subsidiary_authors' not in updated:
                        if 'keywords' in updated:
                            updated['keywords'].append('Zonder auteur')
                        else:
                            updated['keywords'] = ['Zonder auteur']
                    
                    updated_ris.append(updated)
                    status.append('success')
                except Exception as e:
                    print(e)
                    ris['label'] = f'failure ({str(e)})'
                    updated_ris.append(ris)
                    status.append('failure')
            else:
                ris['label'] = 'failure'
                updated_ris.append(ris)
                status.append('failure')

        # store the newly merged information as a JSON string that holds a RIS entry:
        df['consolidated'] = [json.dumps(r, indent=2, ensure_ascii=False) for r in updated_ris]
        df['status'] = status

        # re-encode the original RIS entry as a JSON string in the original column:
        df['RIS'] = [json.dumps(d, indent=2, ensure_ascii=False) for d in df['RIS']]

        # remove the cleaned bibtex string:
        del df['bibtex-clean']

        # output new spreadsheet:
        df.to_excel(spreadsheet_path, index=False, header=True)

        # Ensure that 'extra' field is correctly set as a list for each record
        for record in updated_ris:
            if 'extra' in record and not isinstance(record['extra'], str):
                record['extra'] = ' /// '.join(record['extra'])

        # output updated RIS file:
        with open(f'{decade_folder}/{ptype}_consolidated.ris', 'w') as bibliography_file:
            rispy.dump(updated_ris, bibliography_file, mapping=mappings)

        # show the failure statistics:
        print(df['status'].value_counts())

::: ../data/llm-dump/1940s :::
     -  ../data/llm-dump/1940s/BOOK.xlsx (BOOK)


  9%|▊         | 124/1442 [00:00<00:01, 1235.20it/s]

list index out of range
list index out of range
list index out of range


 18%|█▊        | 256/1442 [00:00<00:00, 1284.68it/s]

list index out of range


...

syntax error in line 8: premature end of file
syntax error in line 8: premature end of file


100%|██████████| 3483/3483 [00:02<00:00, 1393.87it/s]


status
success    3480
failure       3
Name: count, dtype: int64
     -  ../data/llm-dump/misc/WEB.xlsx (WEB)


100%|██████████| 32/32 [00:00<00:00, 1381.28it/s]

Too many commas in 'Wim van Anrooij, Ingrid Biesheuvel, Karina van Dalen-Oskam, Jan Noordegraaf'
status
success    31
failure     1
Name: count, dtype: int64


Extract journal titles for which we don't have a normalization yet and map them provionally to the closest available normalized title (using the Levenshtein distance):

In [13]:
#mappings = []
#for nj, cnt in new_jtitles.items():
#    distances = np.array([lev.distance(nj, oj) for oj in jtitle['normalized']])
#    mappings.append([nj, cnt] + list(jtitle.iloc[np.argmin(distances)][['normalized', 'issn']]))

#mappings = pd.DataFrame(mappings, columns=['raw title', 'count', 'normalized', 'issn'])
#mappings = mappings.sort_values('count', ascending=False)
#mappings.head(30)

We save this spreadsheet for manual correction:

In [14]:
#mappings.to_excel('../data/journal_titles_2ndBatch.xlsx', header=True, index=False)